In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import shelve
import random
import cv2 as cv
import matplotlib.pyplot as plt

import importlib
import model_1
_ = importlib.reload(model_1)

In [2]:
DEVICE = 'cuda'
BATCH_SIZE = 16
SHELVE_PATH = 'data/processed-data/data-1/db'

## Data Preparation

In [3]:
ds = model_1.ShelveDataset(SHELVE_PATH)
dl = DataLoader(
    ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=model_1.ShelveDataset.collate_fn, 
    num_workers=8,
)

## Training Loop

In [4]:
model = model_1.AvgPoolingCnn().to(DEVICE)

In [5]:
# 1) Set up loss function and optimizer
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train() # Set model to train mode (turn on gradient tracking)

epochs = 100
for epoch in range(epochs):
	for batch_idx, (X_train, y_train) in enumerate(dl):
		for k, v in X_train.items():
			X_train[k] = v.to(DEVICE)
		for k, v in y_train.items():
			y_train[k] = v.to(DEVICE)

		# 2) Forward Propagation
		y_pred = model(X_train)
		# break

		# 3) loss calculation
		curr_loss = loss(y_pred, y_train["SCS"])

		# 4) Clear out optimizer gradients
		optimizer.zero_grad()

		# 5) Back Propagation
		curr_loss.backward()

		# 6) Optimizer step
		optimizer.step()

		print(f"[Epoch {f'{epoch+1}/{epochs}':<7}][Batch {batch_idx:<3}]\tLoss: {curr_loss}")

[Epoch 1/100  ][Batch 0  ]	Loss: 7.463075160980225
[Epoch 1/100  ][Batch 1  ]	Loss: 54.79018020629883
[Epoch 1/100  ][Batch 2  ]	Loss: 66.20771789550781
[Epoch 1/100  ][Batch 3  ]	Loss: 84.72021484375
[Epoch 1/100  ][Batch 4  ]	Loss: 100.67752075195312
[Epoch 1/100  ][Batch 5  ]	Loss: 129.2393035888672
[Epoch 1/100  ][Batch 6  ]	Loss: 112.7170639038086
[Epoch 1/100  ][Batch 7  ]	Loss: 114.82838439941406
[Epoch 1/100  ][Batch 8  ]	Loss: 97.14972686767578
[Epoch 1/100  ][Batch 9  ]	Loss: 72.88104248046875
[Epoch 1/100  ][Batch 10 ]	Loss: 212.95040893554688
[Epoch 1/100  ][Batch 11 ]	Loss: 46.761695861816406
[Epoch 1/100  ][Batch 12 ]	Loss: 60.223731994628906
[Epoch 1/100  ][Batch 13 ]	Loss: 51.31937789916992
[Epoch 1/100  ][Batch 14 ]	Loss: 82.19158172607422
[Epoch 1/100  ][Batch 15 ]	Loss: 51.464229583740234
[Epoch 1/100  ][Batch 16 ]	Loss: 52.60686492919922
[Epoch 1/100  ][Batch 17 ]	Loss: 76.40847778320312
[Epoch 1/100  ][Batch 18 ]	Loss: 294.6927185058594
[Epoch 1/100  ][Batch 19 ]	L

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 23.57 GiB of which 59.12 MiB is free. Process 15890 has 53.91 MiB memory in use. Process 28457 has 1.22 GiB memory in use. Including non-PyTorch memory, this process has 18.74 GiB memory in use. Of the allocated memory 17.57 GiB is allocated by PyTorch, and 873.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
# Save Model
torch.save(model, 'models/SCS-model-dict.pt')
